# Algorithms

[Click here to run this chapter on Colab](https://colab.research.google.com/github/AllenDowney/DSIRP/blob/main/notebooks/algorithms.ipynb)

## Searching for anagrams

In this notebook we'll implement algorithms for two tasks:

* Testing a pair of words to see if they are anagrams of each other, that is, if you can rearrange the letters in one word to spell the other.

* Searching a list of words for all pairs that are anagrams of each other.

There is a point to these examples, which I will explain at the end.

**Exercise 1:** Write a function that takes two words and returns `True` if they are anagrams. Test your function with the examples below.

In [4]:
def is_anagram(word1, word2):
    return sorted(word1.lower()) == sorted(word2.lower())

In [5]:
is_anagram('tachymetric', 'mccarthyite') # True

True

In [6]:
is_anagram('post', 'top') # False, letter not present

False

In [7]:
is_anagram('pott', 'top') # False, letter present but not enough copies

False

In [8]:
is_anagram('top', 'post') # False, letters left over at the end

False

In [9]:
is_anagram('topss', 'postt') # False

False

**Exercise 2:** Use `timeit` to see how fast your function is for these examples:

In [10]:
%timeit is_anagram('tops', 'spot')

835 ns ± 215 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
%timeit is_anagram('tachymetric', 'mccarthyite')

1.18 µs ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


How can we compare algorithms running on different computers?

## Searching for anagram pairs

**Exercise 3:** Write a function that takes a word list and returns a list of all anagram pairs.

In [12]:
short_word_list = ['proudest', 'stop', 'pots', 'tops', 'sprouted']

In [13]:
def all_anagram_pairs(word_list):
    anagram_pair_list = []
    for word1 in word_list:
        for word2 in word_list:
            if word1!=word2 and is_anagram(word1, word2) and word1 not in anagram_pair_list:
              anagram_pair_list.append(word1)
    return anagram_pair_list

In [14]:
all_anagram_pairs(short_word_list)

['proudest', 'stop', 'pots', 'tops', 'sprouted']

The following cell downloads a file containing a list of English words.

In [15]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/DSIRP/raw/main/american-english')

The following function reads a file and returns a set of words (I used a set because after we convert words to lower case, there are some repeats.)

In [16]:
def read_words(filename):
    """Read lines from a file and split them into words."""
    res = set()
    for line in open(filename):
        for word in line.split():
            res.add(word.strip().lower())
    return res

In [17]:
word_list = read_words('american-english')
len(word_list)

100781

**Exercise 4:** Loop through the word list and print all words that are anagrams of `stop`.

In [18]:
for word in word_list:
  if is_anagram(word, 'stop'):
    print(word)

pots
stop
spot
opts
tops
post


Now run `all_anagram_pairs` with the full `word_list`:

In [19]:
pairs = all_anagram_pairs(word_list)

KeyboardInterrupt: 

**Exercise 5:** While that's running, let's estimate how long it's going to take.

In [20]:
%timeit all_anagram_pairs(word_list)

KeyboardInterrupt: 

## A better algorithm

**Exercise 6:** Write a better algorithm! Hint: make a dictionary. How much faster is your algorithm?

In [21]:
def all_anagram_lists(word_list):
    """Finds all anagrams in a list of words.
    word_list: sequence of strings
    """
    d = {}
    for word in word_list:
        t = tuple(sorted(word))
        if t not in d:
            d[t] = [word]
        else:
            d[t].append(word)

    # return only the groups with more than one word
    res = {}
    for key, val in d.items():
        if len(val) > 1:
            res[key] = val
    return res

In [22]:
%time anagram_map = all_anagram_lists(word_list)

CPU times: user 407 ms, sys: 22.9 ms, total: 430 ms
Wall time: 433 ms


In [23]:
len(anagram_map)

5872

## Summary

What is the point of the examples in this notebook?

* The different versions of `is_anagram` show that, when inputs are small, it is hard to say which algorithm will be the fastest. It often depends on details of the implementation. Anyway, the differences tend to be small, so it might not matter much in practice.

* The different algorithms we used to search for anagram pairs show that, when inputs are large, we can often tell which algorithm will be fastest. And the difference between a fast algorithm and a slow one can be huge!

## Exercises

Before you work on these exercises, you might want to read the Python [Sorting How-To](https://docs.python.org/3/howto/sorting.html). It uses `lambda` to define an anonymous function, which [you can read about here](https://www.w3schools.com/python/python_lambda.asp).

**Exercise 7:**
Make a dictionary like `anagram_map` that contains only keys that map to a list with more than one element. You can use a `for` loop to make a new dictionary, or a [dictionary comprehension](https://www.freecodecamp.org/news/dictionary-comprehension-in-python-explained-with-examples/).

In [24]:
anagram_map = {}
for word in word_list:
  t = tuple(sorted(word))
  if t not in anagram_map:
    anagram_map[t] = [word]
  else:
    anagram_map[t].append(word)

**Exercise 8:**
Find the longest word with at least one anagram. Suggestion: use the `key` argument of `sort` or `sorted` ([see here](https://stackoverflow.com/questions/8966538/syntax-behind-sortedkey-lambda)).

In [25]:
anagram_map_with_pairs = {key: value for key, value in anagram_map.items() if len(value) > 1}
# Find the longest word among all words that have at least one anagram
# We can iterate through the values (lists of anagrams) and find the maximum length word
longest_word_with_anagram = max([word for word_list in anagram_map_with_pairs.values() for word in word_list], key=len)

print(f"The longest word with at least one anagram is: {longest_word_with_anagram}")

The longest word with at least one anagram is: permissiveness's


**Exercise 9:**
Find the largest list of words that are anagrams of each other.

In [28]:
# Find the key of the largest list of words that are anagrams of each other
largest_list_key = max(anagram_map, key=lambda k: len(anagram_map[k]))
# Print the list of words with the key largest_list_key
print(f"The largest list of words that are anagrams of each other is: {anagram_map[largest_list_key]}")

The largest list of words that are anagrams of each other is: ['stale', 'slate', 'tesla', 'stael', 'least', 'teals', 'tales', 'steal']


**Exercise 10:**
Write a function that takes an integer `word_length` and finds the longest list of words with the given length that are anagrams of each other.

In [34]:
def longest_anagram_list(word_length):
  # Sort anagram_map keeping only the items of key with length = word_length
  a = {key: value for key, value in anagram_map.items() if len(key) == word_length}
  # Find the index of the longest list of words
  m = max(a, key=lambda k: len(a[k]))
  return a[m]

In [35]:
print(longest_anagram_list(4))

['sale', 'ales', 'seal', 'lesa', 'leas', 'elsa']


**Exercise 11:**
At this point we have a data structure that contains lists of words that are anagrams, but we have not actually enumerated all pairs.
Write a function that takes `anagram_map` and returns a list of all anagram pairs.
How many are there?

In [38]:
def anagram_pairs_list(anagram_map):
  # Create a list of all anagram pairs
  anagram_pairs = []
  for key, value in anagram_map.items():
    for i in range(len(value)):
      for j in range(i+1, len(value)):
        anagram_pairs.append((value[i], value[j]))
  return anagram_pairs

In [39]:
print(anagram_pairs_list(anagram_map))

[('listened', 'tinseled'), ('listened', 'enlisted'), ('tinseled', 'enlisted'), ('persian', 'paniers'), ('tocsin', 'tonics'), ("walsh's", "shawl's"), ('lynda', 'dylan'), ('snips', 'spins'), ('flower', 'fowler'), ('deli', 'idle'), ('deli', 'lied'), ('idle', 'lied'), ("pal's", "lap's"), ('sent', 'nets'), ('sent', 'tens'), ('sent', 'nest'), ('nets', 'tens'), ('nets', 'nest'), ('tens', 'nest'), ('armrest', 'smarter'), ('deserts', 'dessert'), ('traduce', 'decatur'), ('tropes', 'poster'), ('tropes', 'presto'), ('poster', 'presto'), ('gina', 'gain'), ('gina', 'agni'), ('gain', 'agni'), ("veal's", "elva's"), ("veal's", "vela's"), ("veal's", "vale's"), ("elva's", "vela's"), ("elva's", "vale's"), ("vela's", "vale's"), ('deform', 'formed'), ('sowed', 'dowse'), ('opal', 'alpo'), ('misleads', 'misdeals'), ('fairs', 'farsi'), ("sumter's", "muster's"), ("martel's", "armlet's"), ("lear's", "real's"), ("lear's", "earl's"), ("real's", "earl's"), ('frayed', 'defray'), ('sparing', 'parings'), ('sparing', '

In [40]:
print(len(anagram_pairs_list(anagram_map)))

9396


*Data Structures and Information Retrieval in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)